In [2]:
import pandas as pd
import numpy as np

1, What is the best way to manage memory?
2, Is merging to one big dataframe the best way to work?
3, Noticing that there are significantly more tv related data rather than movies. In terms of the project, would it be better to keep this specific to movies?
4, There are only 952239 different ratings available, would it be better to drop all other films without a rating?
5, Which features should I keep and which should I remove?  Should I get rid of these during data import or after I've at least done some EDA?


## Data import

### title.basics.tsv

In [3]:
#setting option to display full content rather than truncating at 50 characters
pd.set_option('display.max_colwidth', -1)

#Import the title.basics.tsv file without accounting for any null values to see how the data looks like
bas = 'data/title.basics.tsv'
df_b = pd.read_csv(bas,delimiter = '\t')

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(df_b.tail())
#Around 6M records are present

print(df_b.columns)
#There are 9 columns in the file

#.info() is not showing us the NaN counts, why?
print(df_b.info())
print(df_b.shape)



            tconst  titleType               primaryTitle  \
6000318  tt9916848  tvEpisode  Episode #3.17               
6000319  tt9916850  tvEpisode  Episode #3.19               
6000320  tt9916852  tvEpisode  Episode #3.20               
6000321  tt9916856  short      The Wind                    
6000322  tt9916880  tvEpisode  Horrid Henry Knows It All   

                     originalTitle  isAdult startYear endYear runtimeMinutes  \
6000318  Episode #3.17              0        2010      \N      \N              
6000319  Episode #3.19              0        2010      \N      \N              
6000320  Episode #3.20              0        2010      \N      \N              
6000321  The Wind                   0        2015      \N      27              
6000322  Horrid Henry Knows It All  0        2014      \N      \N              

                          genres  
6000318  Action,Drama,Family      
6000319  Action,Drama,Family      
6000320  Action,Drama,Family      
6000321  Short    

In [5]:
#counting the null values for each columns
print(df_b.isna().sum())
#We can see there are a few NaN values for primaryTitle, originalTitle, genres columns

#locating each NaN values in those three columns above
print(df_b['genres'].loc[df_b.isna()['genres']==True])
print(df_b['primaryTitle'].loc[df_b.isna()['primaryTitle']==True])
print(df_b['originalTitle'].loc[df_b.isna()['originalTitle']==True])

tconst            0
titleType         0
primaryTitle      7
originalTitle     7
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            8
dtype: int64
1102467    NaN
2545024    NaN
3270214    NaN
5020249    NaN
5441686    NaN
5955419    NaN
5992291    NaN
5996656    NaN
Name: genres, dtype: object
2199580    NaN
2292206    NaN
3459094    NaN
4068855    NaN
5317026    NaN
5348809    NaN
5956130    NaN
Name: primaryTitle, dtype: object
2199580    NaN
2292206    NaN
3459094    NaN
4068855    NaN
5317026    NaN
5348809    NaN
5956130    NaN
Name: originalTitle, dtype: object


We can see the NaN values for primaryTitle and originalTitle are the same.
Also, in the actual dataframe, we see quite a few of '\N' values.  

According to the data documentation at https://www.imdb.com/interfaces/, 'A ‘\N’ is used to denote that a particular field is missing or null for that title/name.'

df_b = pd.read_csv(bas,delimiter = '\t',na_values='\\N')

This will be determined after we look at further datasets.


In [6]:
#unlike originalTitle or primaryTitle, genres column is very important for our analysis.
#so we will look at the 8 records that genres column is NaN for and possibly drop them.
print(df_b.loc[df_b['genres'].isna() == True])


             tconst  titleType  \
1102467  tt10233364  tvEpisode   
2545024  tt2347742   tvEpisode   
3270214  tt3984412   tvEpisode   
5020249  tt7841930   tvEpisode   
5441686  tt8740950   tvEpisode   
5955419  tt9822816   tvEpisode   
5992291  tt9900062   tvEpisode   
5996656  tt9909210   tvEpisode   

                                                                                                                                                                   primaryTitle  \
1102467  Rolling in the Deep Dish\tRolling in the Deep Dish                                                                                                                       
2545024  No sufras por la alergia esta primavera\tNo sufras por la alergia esta primavera                                                                                         
3270214  I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race\tI'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race       

Here we see something does not add up.  isAdult column is showing year and runtimeMinutes column is showing data similar to genres.   Looking deeper into this, we can see the primaryTitle contains '\t' that has not been interpreted as delimiter.  This would be a parsing issue rather than missing data.


In [7]:
#grabbing indexes for null genre records
gen_ind = df_b.loc[df_b['genres'].isna()==True].index
print(gen_ind)

Int64Index([1102467, 2545024, 3270214, 5020249, 5441686, 5955419, 5992291,
            5996656],
           dtype='int64')


When checking the data file directly, we can see that the reason \t wasn't considered as a delimiter was due to quotation mark.  
For example, "Rolling in the Deep Dish	"Rolling in the Deep Dish   
Above was seen for primartyTitle and originalTitle columns for tconst, tt10233364 
This is the case for the all the rest of the records.  Since writing an algorithm to revise, save to file and reimport as dataframe would take longer time than manual fix, I will make a copy of the file and manually remove those quotation marks.

new file: title.basics_0.tsv



In [8]:
bas0 = 'data/title.basics_0.tsv'
df_b0 = pd.read_csv(bas0,delimiter = '\t')

In [9]:
print(df_b0.tail())
#Around 6M records are present

print(df_b0.columns)
#There are 9 columns in the file

#.info() is not showing us the NaN counts, why?
print(df_b0.info())
print(df_b0.shape)

            tconst  titleType               primaryTitle  \
6000318  tt9916848  tvEpisode  Episode #3.17               
6000319  tt9916850  tvEpisode  Episode #3.19               
6000320  tt9916852  tvEpisode  Episode #3.20               
6000321  tt9916856  short      The Wind                    
6000322  tt9916880  tvEpisode  Horrid Henry Knows It All   

                     originalTitle  isAdult startYear endYear runtimeMinutes  \
6000318  Episode #3.17              0        2010      \N      \N              
6000319  Episode #3.19              0        2010      \N      \N              
6000320  Episode #3.20              0        2010      \N      \N              
6000321  The Wind                   0        2015      \N      27              
6000322  Horrid Henry Knows It All  0        2014      \N      \N              

                          genres  
6000318  Action,Drama,Family      
6000319  Action,Drama,Family      
6000320  Action,Drama,Family      
6000321  Short    

In [10]:
#counting the null values for each columns
print(df_b0.isna().sum())
#We can see there are a few NaN values for primaryTitle, originalTitle, genres columns

#locating each NaN values in those three columns above
print(df_b0['genres'].loc[df_b0.isna()['genres']==True])
print(df_b0['primaryTitle'].loc[df_b0.isna()['primaryTitle']==True])
print(df_b0['originalTitle'].loc[df_b0.isna()['originalTitle']==True])

tconst            0
titleType         0
primaryTitle      7
originalTitle     7
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64
Series([], Name: genres, dtype: object)
2199580    NaN
2292206    NaN
3459094    NaN
4068855    NaN
5317026    NaN
5348809    NaN
5956130    NaN
Name: primaryTitle, dtype: object
2199580    NaN
2292206    NaN
3459094    NaN
4068855    NaN
5317026    NaN
5348809    NaN
5956130    NaN
Name: originalTitle, dtype: object


After the quotation fix, there are no NaN values for genres.

In [11]:
#list out the records where primaryTitle and originalTitle are Nan
print(df_b0.loc[df_b0['primaryTitle'].isna()==True])

#looks like these can be kept for now since there are no parsing error

            tconst  titleType primaryTitle originalTitle  isAdult startYear  \
2199580  tt1971246  tvEpisode  NaN          NaN           0        2011       
2292206  tt2067043  tvEpisode  NaN          NaN           0        1965       
3459094  tt4404732  tvEpisode  NaN          NaN           0        2015       
4068855  tt5773048  tvEpisode  NaN          NaN           0        2015       
5317026  tt8473688  tvEpisode  NaN          NaN           0        \N         
5348809  tt8541336  tvEpisode  NaN          NaN           0        2018       
5956130  tt9824302  tvEpisode  NaN          NaN           0        2016       

        endYear runtimeMinutes              genres  
2199580  \N      \N             Biography           
2292206  \N      \N             Music               
3459094  \N      \N             Comedy              
4068855  \N      \N             Talk-Show           
5317026  \N      \N             Drama               
5348809  \N      \N             Reality-TV,Romanc

In [12]:
#Now that initial NaN was taken care of, we will reimport the file with '\N' taken care of.
df_b0 = pd.read_csv(bas0,delimiter = '\t',na_values='\\N')

#counting the null values for each columns
print(df_b0.isna().sum())

#Now there are quite a few null values present.  This is good enough for now. 

#delete df_b to release memory
del df_b

tconst            0      
titleType         0      
primaryTitle      7      
originalTitle     182    
isAdult           0      
startYear         336604 
endYear           5949470
runtimeMinutes    4206286
genres            477616 
dtype: int64


### title.crew.tsv

Now we will look at the crew file.

In [13]:
df_crew = pd.read_csv('data/title.crew.tsv',delimiter='\t')

print(df_crew.tail())
#Around 6M records are present

print(df_crew.columns)
#There are 3 columns in the file

#.info() is not showing us the NaN counts, why?
print(df_crew.info())
print(df_crew.shape)

#counting the null values for each columns
print(df_crew.isna().sum())

            tconst            directors                        writers
6000318  tt9916848  nm5519454,nm5519375  nm6182221,nm1628284,nm2921377
6000319  tt9916850  nm5519375,nm5519454  nm6182221,nm1628284,nm2921377
6000320  tt9916852  nm5519375,nm5519454  nm6182221,nm1628284,nm2921377
6000321  tt9916856  nm10538645           nm6951431                    
6000322  tt9916880  nm0996406            nm1482639,nm2586970          
Index(['tconst', 'directors', 'writers'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000323 entries, 0 to 6000322
Data columns (total 3 columns):
tconst       object
directors    object
writers      object
dtypes: object(3)
memory usage: 137.3+ MB
None
(6000323, 3)
tconst       0
directors    0
writers      0
dtype: int64


In [14]:
#delete df_crew to release memory
del df_crew

In [15]:
#no NaN found during initial parsing.  Now we will import again to account for '\N'
df_crew0 = pd.read_csv('data/title.crew.tsv',delimiter='\t',na_values='\\N')

#counting the null values for each columns
print(df_crew0.isna().sum())

#example of what they look like
print(df_crew0.loc[df_crew0['directors'].isna()==True].head())

tconst       0      
directors    2499232
writers      2940441
dtype: int64
        tconst directors writers
37   tt0000039  NaN       NaN   
102  tt0000104  NaN       NaN   
123  tt0000125  NaN       NaN   
132  tt0000134  NaN       NaN   
160  tt0000162  NaN       NaN   


### title.principals.tsv

In [16]:
df_pr = pd.read_csv('data/title.principals.tsv',delimiter='\t')

print(df_pr.tail())
#Around 34M records are present

print(df_pr.columns)
#There are 6 columns in the file

#.info() is not showing us the NaN counts, why?
print(df_pr.info())
print(df_pr.shape)

#counting the null values for each columns
print(df_pr.isna().sum())

             tconst  ordering     nconst  category                 job  \
34602926  tt9916880  5         nm0996406  director  principal director   
34602927  tt9916880  6         nm1482639  writer    \N                   
34602928  tt9916880  7         nm2586970  writer    books                
34602929  tt9916880  8         nm1594058  producer  producer             
34602930  tt9916880  9         nm2676923  actress   \N                   

                                                    characters  
34602926  \N                                                    
34602927  \N                                                    
34602928  \N                                                    
34602929  \N                                                    
34602930  ["Sour Susan","Goody-Goody Gordon","Singing Soraya"]  
Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34602931 entries, 0 to 346

In [17]:
#delete df_pr to release memory
del df_pr

In [18]:
#no NaN found during initial parsing.  Now we will import again to account for '\N'
df_pr0 = pd.read_csv('data/title.principals.tsv',delimiter='\t',na_values='\\N')

#counting the null values for each columns
print(df_pr0.isna().sum())

#example of what they look like
print(df_pr0.loc[df_pr0['job'].isna()==True].head())

tconst        0       
ordering      0       
nconst        0       
category      0       
job           28879352
characters    16950162
dtype: int64
      tconst  ordering     nconst  category  job   characters
0  tt0000001  1         nm1588970  self      NaN  ["Herself"]
1  tt0000001  2         nm0005690  director  NaN  NaN        
3  tt0000002  1         nm0721526  director  NaN  NaN        
4  tt0000002  2         nm1335271  composer  NaN  NaN        
5  tt0000003  1         nm0721526  director  NaN  NaN        


### title.ratings.tsv

In [19]:
df_rate = pd.read_csv('data/title.ratings.tsv',delimiter='\t')

print(df_rate.tail())
#Around 1M records are present

print(df_rate.columns)
#There are 3 columns in the file

#.info() is showing us the NaN counts, why?
print(df_rate.info())
print(df_rate.shape)

#counting the null values for each columns
print(df_rate.isna().sum())

           tconst  averageRating  numVotes
952234  tt9916576  6.4            5       
952235  tt9916578  9.0            10      
952236  tt9916720  4.3            21      
952237  tt9916766  7.0            8       
952238  tt9916778  9.2            6       
Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952239 entries, 0 to 952238
Data columns (total 3 columns):
tconst           952239 non-null object
averageRating    952239 non-null float64
numVotes         952239 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 21.8+ MB
None
(952239, 3)
tconst           0
averageRating    0
numVotes         0
dtype: int64


In [20]:
#no NaN found during initial parsing.  Now we will import again to account for '\N'
df_rate0 = pd.read_csv('data/title.ratings.tsv',delimiter='\t',na_values='\\N')

#counting the null values for each columns
print(df_rate0.isna().sum())

#even after accounting for \N values, there are no NaN

tconst           0
averageRating    0
numVotes         0
dtype: int64


### name.basics.tsv

In [21]:
df_name = pd.read_csv('data/name.basics.tsv',delimiter='\t')

print(df_name.tail())
#Around 9M records are present

print(df_name.columns)
#There are 6 columns in the file

#.info() is showing us the NaN counts, why?
print(df_name.info())
print(df_name.shape)

#counting the null values for each columns
print(df_name.isna().sum())

            nconst         primaryName birthYear deathYear primaryProfession  \
9444334  nm9993715  Stephen Crowe       \N        \N        art_director       
9444335  nm9993716  Essias Loberg       \N        \N        NaN                
9444336  nm9993717  Harikrishnan Rajan  \N        \N        cinematographer    
9444337  nm9993718  Aayush Nair         \N        \N        cinematographer    
9444338  nm9993719  Andre Hill          \N        \N        NaN                

        knownForTitles  
9444334  tt2455546      
9444335  \N             
9444336  tt8736744      
9444337  \N             
9444338  \N             
Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9444339 entries, 0 to 9444338
Data columns (total 6 columns):
nconst               object
primaryName          object
birthYear            object
deathYear            object
primaryProfession 

In [22]:
#checking the actual data to see if there are any parsing mistakes
print(df_name.loc[df_name.isna()['primaryProfession']==True].tail())

            nconst        primaryName birthYear deathYear primaryProfession  \
9444329  nm9993710  Nestor Rudnytskyy  \N        \N        NaN                
9444330  nm9993711  David Gluzman      \N        \N        NaN                
9444331  nm9993712  Corny O'Connell    \N        \N        NaN                
9444335  nm9993716  Essias Loberg      \N        \N        NaN                
9444338  nm9993719  Andre Hill         \N        \N        NaN                

        knownForTitles  
9444329  \N             
9444330  \N             
9444331  \N             
9444335  \N             
9444338  \N             


In [23]:
#looks like parsing is done correctly and there are just two different types of null values for this dataset.
#importing again to account for \N
df_name0 = pd.read_csv('data/name.basics.tsv',delimiter='\t',na_values='\\N')

#counting the null values for each columns
print(df_name0.isna().sum())

#delete df_name to free memory
del df_name

nconst               0      
primaryName          0      
birthYear            8960088
deathYear            9275749
primaryProfession    1779417
knownForTitles       1471593
dtype: int64


## Data extraction

df_b0, df_crew0, df_pr0, df_rate0, df_name0 available but we do not need all columns of these dataframes.
So we will strip them down. As well as remove unrelated data.  We are only considering movie related data so film that is not categorizes as movies will be removed.

In [24]:
#listing all columns for b0 dataframe
print(df_b0.columns)

#it looks like there are different types of films other than movies or shorts such as tv series.
print(df_b0.titleType.unique())

#counting for movie type file only
print('There are '+ str(df_b0.titleType.loc[df_b0.titleType=='movie'].count())+' movies out of 6M')
#there are 523864 movies in 6M records

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
['short' 'movie' 'tvMovie' 'tvSeries' 'tvEpisode' 'tvShort' 'tvMiniSeries'
 'tvSpecial' 'video' 'videoGame']
There are 523864 movies out of 6M


In [25]:
#out of these columns, we do not need the 'endYear' and only one of 'primaryTitle' or 'originalTitle'
#looking at a random sample of the two titles, original contains the original language and primary has been mostly translated into English
df_b0.primaryTitle[100:105]
df_b0.originalTitle[100:105]

del df_b0['originalTitle']
del df_b0['endYear']

df_b0.columns

Index(['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres'],
      dtype='object')

In [26]:

#counting for movies
print('There are '+ str(df_b0.titleType.loc[df_b0.titleType=='movie'].count())+' shorts out of 6M')

#counting for shorts
print('There are '+ str(df_b0.titleType.loc[df_b0.titleType=='short'].count())+' shorts out of 6M')

#counting for tv related films
tv_related = df_b0.titleType.loc[df_b0.titleType =='tvMovie'].count() + df_b0.titleType.loc[df_b0.titleType =='tvSeries'].count() + df_b0.titleType.loc[df_b0.titleType =='tvEpisode'].count() + df_b0.titleType.loc[df_b0.titleType =='tvShort'].count() + df_b0.titleType.loc[df_b0.titleType =='tvMiniSeries'].count() + df_b0.titleType.loc[df_b0.titleType =='tvSpecial'].count()
print('There are ' + str(tv_related)+' tv related films out of 6M')

#counting for video
print('There are '+ str(df_b0.titleType.loc[df_b0.titleType=='video'].count())+' videos out of 6M')

#counting for videogames
print('There are '+ str(df_b0.titleType.loc[df_b0.titleType=='videoGame'].count())+' video games out of 6M')

There are 523864 shorts out of 6M
There are 691981 shorts out of 6M
There are 4525184 tv related films out of 6M
There are 235612 videos out of 6M
There are 23682 video games out of 6M


In [27]:
#Now we will remove the rows containing any films other than movies

df_b0 = df_b0[df_b0['titleType']=='movie']

print(df_b0.shape)

(523864, 7)


Now df_b0 only contains records related to movies.  We can use the tconst identifier to remove unneccesary rows in other dataframes.

In [28]:
#next, we look at df_crew0
print(df_crew0.columns)

#we need all columns in this dataframe so we'll simply use the tconst from df_b0 to remove all other tconst rows from df_crew0

#create a list of tconst that we want to extract
tconst = list(df_b0.tconst)

print(df_crew0.shape)
df_crew0=df_crew0[df_crew0['tconst'].isin(tconst)]

print(df_crew0.shape)

Index(['tconst', 'directors', 'writers'], dtype='object')
(6000323, 3)
(523864, 3)


Now we've extracted the movie data out of df_crew0. It also contains the same number of rows as df_b0

In [29]:
#next we look at df_pr0
print(df_pr0.columns)

#we will keep all columns here as well for now as they may become useful

#removing all rows not related to movies
print(df_pr0.shape)
df_pr0=df_pr0[df_pr0['tconst'].isin(tconst)]

print(df_pr0.shape)

print(len(df_pr0.tconst.unique()))

Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
(34602931, 6)
(3724753, 6)
511843


Now df_pr0 contains only tconst related to movie.  This dataframe may contains multiple rows for a single tconst hence the difference in the shape of the dataframe compared to other ones.

In [30]:
#next we look at df_rate0.  We also want to keep all columns here.
print(df_rate0.columns)

#removing all rows not related to movies
print(df_rate0.shape)
df_rate0=df_rate0[df_rate0['tconst'].isin(tconst)]

print(df_rate0.shape)

Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
(952239, 3)
(236124, 3)


In [31]:
#df_name0 has information about the crew rather than the movie so this dataframe does not contain tconst.
#however, there are unneeded columns that we can get rid of.

print(df_name0.columns)
df_name0[1:100]

#we will remove birthYear, deathYear, and knownForTitles columns

del df_name0['birthYear']
del df_name0['deathYear']
del df_name0['knownForTitles']

print(df_name0.shape)
print(df_name0.columns)

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
(9444339, 3)
Index(['nconst', 'primaryName', 'primaryProfession'], dtype='object')


Now that all dataframe contain the rows and columns that are needed, it is time to deal with null values.

In [32]:
#counting the null values for each columns
print(df_b0.shape)
print(df_b0.isna().sum())

print(df_crew0.shape)
print(df_crew0.isna().sum())

print(df_pr0.shape)
print(df_pr0.isna().sum())

print(df_rate0.shape)
print(df_rate0.isna().sum())

print(df_name0.shape)
print(df_name0.isna().sum())

(523864, 7)
tconst            0     
titleType         0     
primaryTitle      0     
isAdult           0     
startYear         60126 
runtimeMinutes    194649
genres            71824 
dtype: int64
(523864, 3)
tconst       0     
directors    48751 
writers      133274
dtype: int64
(3724753, 6)
tconst        0      
ordering      0      
nconst        0      
category      0      
job           3097386
characters    2461715
dtype: int64
(236124, 3)
tconst           0
averageRating    0
numVotes         0
dtype: int64
(9444339, 3)
nconst               0      
primaryName          0      
primaryProfession    1779417
dtype: int64


 Unfortunately, out of 523864 movie related data, there are only ratings available for 236124 of the movies.  To go forward, we have stripped down the dataframe to only contain data with rating value available for all dataframes.  This is because we cannot fill the null values for ratings.  This is the same for missing directors, writers, cast and crews. 

In [40]:
#merging the dataframe to master

#we will create a single dataframe that contains all information for df_b0, df_crew0.
#df_name0 will not be included in the master dataframe as this doest not contain the tconst column.
#df_rate0 can be included with in this master dataframe but this will create null values where we will need to remove some needed records.
#so we will keep df_rate0 separate for now until merging is neccessary.

#df_pr0 contains more than one records per tconst. Meaning this will create duplicate tconst entries that we can fill the values for,
#but this will also make the initial EDA difficult. So we will keep this separate for now as well.

df_master= pd.merge(df_b0,df_crew0,on='tconst')
df_master
print(df_b0.shape)
print(df_crew0.shape)
print(df_master.shape)

print(df_b0.columns)
print(df_crew0.columns)
print(df_master.columns)

print(df_pr0.shape)
print(df_pr0.columns)
print(df_rate0.shape)

(523864, 7)
(523864, 3)
(523864, 9)
Index(['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'directors', 'writers'], dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'directors', 'writers'],
      dtype='object')
(3724753, 6)
Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
(236124, 3)


In [42]:
#Let's save the dataframes to csv to be used in Data Storytelling

df_master.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_master.csv')
df_pr0.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_pr0.csv')
df_rate0.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_rate0.csv')
df_name0.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_name0.csv')